In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from convert_date import *

In [ ]:
# s = requests.Session()
# s.headers.update({
#    "Accept-Language": "en-US,en;q=0.9", 
#    "Accept-Encoding": "gzip, deflate, br",
#    "User-Agent": "Java-http-client/"
# })
# r = s.get("https://www.nasdaq.com/market-activity/quotes/nasdaq-ndx-index")
# soup = BeautifulSoup(r.content, "html.parser")
# res = json.loads([x for x in soup.find("script", {"type": "application/json"})][0])

In [4]:
# headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"}

s = requests.Session()
#headers = {'user-agent': 'Chrome/126.0.6478.127 Edge/126.0.2592.87', 'type': 'application/json'}

headers = {'type': 'application/json'}
res = s.get("https://api.nasdaq.com/api/quote/list-type/nasdaq100", headers=headers)

if res.status_code == 200:

    main_data = res.json()['data']['data']['rows']
    df_index = range(len(main_data))
    
    df_data = pd.DataFrame(columns=['Symbol', 'Company Name', 'Market Cap'], index=df_index)
    for i in df_index:
        md = main_data[i]
        df_data.loc[i, 'Symbol'] = md['symbol']
        df_data.loc[i, 'Company Name'] = md['companyName']
        df_data.loc[i, 'Market Cap'] = md['marketCap']
    
    print(df_data)
# print(md['symbol'], md['companyName'], md['marketCap'])

In [12]:
print(df_data.sort_values(['Market Cap']))

   Symbol                                     Company Name         Market Cap
76   META        Meta Platforms, Inc. Class A Common Stock  1,280,645,382,352
77   PANW            Palo Alto Networks, Inc. Common Stock    108,443,696,100
5    KLAC                     KLA Corporation Common Stock    117,341,410,096
45    ADI                Analog Devices, Inc. Common Stock    118,754,618,217
18   REGN     Regeneron Pharmaceuticals, Inc. Common Stock    121,141,247,563
..    ...                                              ...                ...
60    WBA      Walgreens Boots Alliance, Inc. Common Stock      9,906,081,050
91   CRWD  CrowdStrike Holdings, Inc. Class A Common Stock     90,070,206,100
25   SNPS                      Synopsys, Inc. Common Stock     94,250,879,609
93   ABNB                Airbnb, Inc. Class A Common Stock     94,771,047,377
48    ADP     Automatic Data Processing, Inc. Common Stock     97,401,104,733

[101 rows x 3 columns]


In [2]:
aapl_json = {"symbol":"AAPL","sector":"","companyName":"Apple Inc. Common Stock","marketCap":"3,548,766,597,260","lastSalePrice":"$231.441","netChange":"$3.871","percentageChange":"+1.70%","deltaIndicator":"up"}

In [ ]:
headers = {'type': 'application/json'}
s.get("https://api.nasdaq.com/api/quote/list-type/nasdaq100", headers=headers)